# Практическое задание к уроку 3. Dataset, Dataloader, BatchNorm, Dropout, Оптимизация

Будем практиковаться на датасете:
https://www.kaggle.com/c/avito-demand-prediction

Ваша задача:
1. Создать Dataset для загрузки данных (используем только числовые данные)
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

Вопросы? в личку @Kinetikm

In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm_notebook
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('ggplot')
plt.rcParams["font.family"] = "Times New Roman"

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
train = pd.read_csv('/content/drive/MyDrive/train.csv')
train.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2.0,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19.0,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9.0,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286.0,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3.0,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [ ]:
# train = pd.read_csv('train.csv')
# train.head(2)

item_id - Идентификатор объявления.

user_id - Идентификатор пользователя.

region - Рекламный регион.

city - Рекламный город.

parent_category_name - Категория объявлений верхнего уровня, классифицированная по рекламной модели Avito.

category_name - Категория мелкозернистых объявлений, классифицированная по рекламной модели Avito.

param_1 - Необязательный параметр из рекламной модели Avito.

param_2 - Необязательный параметр из рекламной модели Avito.

param_3 - Необязательный параметр из рекламной модели Avito.

title - Название объявления.

description - Описание объявления.

price - Цена объявления.

item_seq_number - Порядковый номер объявления для пользователя.

activation_date- Дата размещения объявления.

user_type - Тип пользователя.

image - Идентификационный код изображения. Привязка к файлу jpg в train_jpg. Не в каждой рекламе есть изображение.

image_top_1 - Классификационный код Авито для изображения.

deal_probability - Целевая переменная. Это вероятность того, что объявление действительно что-то продало. Невозможно с уверенностью проверить каждую транзакцию, поэтому значение этого столбца может быть любым с плавающей точкой от нуля до единицы.

In [7]:
#  preprocessing
train.activation_date = pd.to_datetime(train.activation_date)
train['day_of_month'] = train.activation_date.apply(lambda x: x.day)
train['day_of_week'] = train.activation_date.apply(lambda x: x.weekday())

# кодирование средним
agg_cols = ['region', 'category_name',
            'city', 'user_type']
for col in tqdm_notebook(agg_cols):
    gp = train.groupby(col)['deal_probability']
    mean = gp.mean()
    train[col + '_deal_probability_avg'] = train[col].map(mean)

train = train.drop(['city', 'category_name', 'user_id', 'description', 'image', 'parent_category_name', 'region',
                    'item_id', 'param_1', 'param_2', 'param_3', 'title', 'user_type', 'activation_date'], axis=1)

for col in train.columns:
    if train[col].isna().sum() > 0:
        # train.drop(columns=col, inplace=True)
        train[col].fillna(train[col].median(), inplace=True)

display(train.head(2), train.info())

  0%|          | 0/4 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218651 entries, 0 to 218650
Data columns (total 10 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   price                               218651 non-null  float64
 1   item_seq_number                     218651 non-null  float64
 2   image_top_1                         218651 non-null  float64
 3   deal_probability                    218651 non-null  float64
 4   day_of_month                        218651 non-null  float64
 5   day_of_week                         218651 non-null  float64
 6   region_deal_probability_avg         218651 non-null  float64
 7   category_name_deal_probability_avg  218651 non-null  float64
 8   city_deal_probability_avg           218651 non-null  float64
 9   user_type_deal_probability_avg      218651 non-null  float64
dtypes: float64(10)
memory usage: 16.7 MB


,price,item_seq_number,image_top_1,deal_probability,day_of_month,day_of_week,region_deal_probability_avg,category_name_deal_probability_avg,city_deal_probability_avg,user_type_deal_probability_avg
0,400.0,2.0,1008.0,0.12789,28.0,1.0,0.124054,0.199964,0.124253,0.150049
1,3000.0,19.0,692.0,0.00000,26.0,6.0,0.131907,0.191021,0.133348,0.150049


None

In [8]:
train, test = train_test_split(train, test_size=0.25, random_state=13)
train.head(2)

,price,item_seq_number,image_top_1,deal_probability,day_of_month,day_of_week,region_deal_probability_avg,category_name_deal_probability_avg,city_deal_probability_avg,user_type_deal_probability_avg
60544,250.0,535.0,2433.0,0.0,24.0,4.0,0.131332,0.052187,0.131003,0.124831
180675,2000.0,36.0,82.0,0.0,28.0,1.0,0.121613,0.061891,0.112069,0.150049


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163988 entries, 60544 to 178352
Data columns (total 10 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   price                               163988 non-null  float64
 1   item_seq_number                     163988 non-null  float64
 2   image_top_1                         163988 non-null  float64
 3   deal_probability                    163988 non-null  float64
 4   day_of_month                        163988 non-null  float64
 5   day_of_week                         163988 non-null  float64
 6   region_deal_probability_avg         163988 non-null  float64
 7   category_name_deal_probability_avg  163988 non-null  float64
 8   city_deal_probability_avg           163988 non-null  float64
 9   user_type_deal_probability_avg      163988 non-null  float64
dtypes: float64(10)
memory usage: 13.8 MB


In [10]:
class DFDataset(torch.utils.data.Dataset):
    def __init__(self, df, normalize=False, fit_scaler=False):
        self.df = df.copy()
        self.normalize = normalize
        self.scaler = MinMaxScaler()
        self.fit_scaler = fit_scaler
        self.sc_fl = 0

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        label = self.df.iloc[idx, -1:]
        if self.fit_scaler:
            self.scaler.fit(df.iloc[:, :-1])
            self.sc_fl = 1

        if self.normalize and sc_fl:
            df = scaler.transform(df)

        tensor = torch.FloatTensor(self.df.iloc[idx, 1:-1].values)
        label = torch.FloatTensor(label.values)

        return tensor, label

In [11]:
train_dataset = DFDataset(train)
test_dataset = DFDataset(test)

In [12]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=2048,
                                           shuffle=True,
                                           num_workers=4)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=2048,
                                          shuffle=True,
                                          num_workers=4)

In [13]:
class FFNet(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FFNet, self).__init__()
        self.bn1 = nn.BatchNorm1d(input_dim)
        self.fc1 = nn.Linear(input_dim, 5*hidden_dim)
        self.dp1 = nn.Dropout(0.40)

        self.bn2 = nn.BatchNorm1d(5*hidden_dim)
        self.fc2 = nn.Linear(5*hidden_dim, 2*hidden_dim)
        self.dp2 = nn.Dropout(0.15)

        self.bn4 = nn.BatchNorm1d(2*hidden_dim)
        self.fc4 = nn.Linear(2*hidden_dim, 1)

    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = F.tanh(x)
        x = self.dp1(x)

        x = self.bn2(x)
        x = self.fc2(x)
        x = F.tanh(x)
        x = self.dp2(x)

        x = self.bn4(x)
        x = self.fc4(x)
        x = F.sigmoid(x)

        return x

In [14]:
def rmsle_loss(y_pred, y_true):
    loss = torch.sqrt(torch.mean((torch.log(y_pred+1)-torch.log(y_true+1))**2))
    return loss

In [15]:
EPOCHES = 1
LR = 0.001

In [16]:
def train_loop(tr_dataloader, ev_dataloader, model, optimizer, history):

    size = len(tr_dataloader.dataset)
    for batch, (X, y) in enumerate(tr_dataloader):
        # Compute prediction and loss
        pred = model(X.to(device))
        loss = rmsle_loss(pred, y.to(device))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 50 == 0:
            current = batch * len(X)
            history['train'].append(loss)
            test_loss = eval_loop(ev_dataloader, model)
            history['eval'].append(test_loss)
            print(f"loss: {loss:>7f}, Avg test loss: {test_loss:>8f}  [{current:>5d}/{size:>5d}]")

    return history


def eval_loop(ev_dataloader, model):
    
    size = len(ev_dataloader.dataset)
    num_batches = len(ev_dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in ev_dataloader:
            pred = model(X.to(device))
            test_loss += rmsle_loss(pred, y.to(device))

    test_loss /= num_batches
    
    return test_loss

### SGD

In [17]:
model = FFNet(8, 5).to(device)

In [18]:
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

In [19]:
history_1 = {'train': [], 'eval': []}

for t in tqdm_notebook(range(EPOCHES)):
    print(f"Epoch {t+1}\n-------------------------------")
    history_1 = train_loop(train_loader,
                           test_loader,
                           model,
                           optimizer,
                           history_1
                          )
print("Done!")

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1
-------------------------------
loss: 0.275895, Avg test loss: 0.276509  [    0/163988]
loss: 0.274678, Avg test loss: 0.274884  [102400/163988]
Done!


### RMSProp

In [20]:
model = FFNet(8, 5).to(device)

In [21]:
optimizer = torch.optim.RMSprop(model.parameters(), lr=LR)

In [22]:
history_2 = {'train': [], 'eval': []}

for t in tqdm_notebook(range(EPOCHES)):
    print(f"Epoch {t+1}\n-------------------------------")
    history_2 = train_loop(train_loader,
                           test_loader,
                           model,
                           optimizer,
                           history_2
                          )
print("Done!")

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1
-------------------------------
loss: 0.308591, Avg test loss: 0.301366  [    0/163988]
loss: 0.207923, Avg test loss: 0.206895  [102400/163988]
Done!


### ADAM

In [23]:
model = FFNet(8, 5).to(device)

In [24]:
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [25]:
history_3 = {'train': [], 'eval': []}

for t in tqdm_notebook(range(EPOCHES)):
    print(f"Epoch {t+1}\n-------------------------------")
    history_3 = train_loop(train_loader,
                           test_loader,
                           model,
                           optimizer,
                           history_3
                          )
print("Done!")

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1
-------------------------------
loss: 0.304663, Avg test loss: 0.303218  [    0/163988]
loss: 0.278250, Avg test loss: 0.277832  [102400/163988]
Done!


### SGD + Momentum

In [26]:
model = FFNet(8, 5).to(device)

In [27]:
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.8)

In [28]:
history_4 = {'train': [], 'eval': []}

for t in tqdm_notebook(range(EPOCHES)):
    print(f"Epoch {t+1}\n-------------------------------")
    history_4 = train_loop(train_loader,
                           test_loader,
                           model,
                           optimizer,
                           history_4
                          )
print("Done!")

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1
-------------------------------
loss: 0.316012, Avg test loss: 0.316450  [    0/163988]
loss: 0.309821, Avg test loss: 0.309865  [102400/163988]
Done!


**Вывод:** проверка скорости схождения при фиксированной скорости обучения показала, что ADAM быстрее всего, SGD - медленее всего. Сохранение градиентов в SGD + Momentum быстрее приводит к минимуму функции, нежели просто SGD.